<a href="https://colab.research.google.com/github/ayulockin/deepgenerativemodeling/blob/master/Hyperparameter_Sweep_Variational_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setups, Imports and Installations

In [0]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
import tensorflow.keras.backend as K

2.2.0-rc2


In [0]:
%%capture
!pip install wandb

import wandb
from wandb.keras import WandbCallback

!wandb login 69f60a7711ce6b8bbae91ac6d15e45d6b1f1430e

In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from tqdm.notebook import tqdm

# Let's use MNIST Dataset

In [0]:
from keras.datasets import mnist

# MNIST dataset
(train_img, train_label), (test_img, test_label) = mnist.load_data()

image_size = train_img.shape[1]
original_dim = image_size * image_size
train_img = np.reshape(train_img, [-1, original_dim])
test_img = np.reshape(test_img, [-1, original_dim])
train_img = train_img.astype('float32') / 255
test_img = test_img.astype('float32') / 255

## Just dataset info
print("X_train: ", train_img.shape)
print("X_test: ", test_img.shape)

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step
X_train:  (60000, 784)
X_test:  (10000, 784)


# Model

In [0]:
class VariationalAutoencoder:
  def __init__(self, input_shape, original_dim, intermediate_dim, latent_dim):
    self.input_shape = input_shape
    self.original_dim = original_dim
    self.intermediate_dim = intermediate_dim
    self.latent_dim = latent_dim

  def build_model(self):
    # instantiate VAE model
    self.build_encoder()
    self.build_decoder()

    self.outputs = self.decoder(self.encoder(self.inputs)[2])

    self.vae = keras.models.Model(self.inputs, self.outputs, name='vae_mlp')
    self.add_loss()

    return self.vae

  def sampling(self, args):
      z_mean, z_log_var = args
      batch = K.shape(z_mean)[0]
      dim = K.int_shape(z_mean)[1]
      # by default, random_normal has mean = 0 and std = 1.0
      epsilon = K.random_normal(shape=(batch, dim))
      return z_mean + K.exp(0.5 * z_log_var) * epsilon

  def build_encoder(self):
    self.inputs = keras.layers.Input(shape=self.input_shape, name='encoder_input')
    x = keras.layers.Dense(self.intermediate_dim, activation='relu')(self.inputs)
    self.z_mean = keras.layers.Dense(self.latent_dim, name='z_mean')(x)
    self.z_log_var = keras.layers.Dense(self.latent_dim, name='z_log_var')(x)

    self.z = keras.layers.Lambda(self.sampling, output_shape=(self.latent_dim,), name='z')([self.z_mean, self.z_log_var])

    # instantiate encoder model
    self.encoder = keras.models.Model(self.inputs, [self.z_mean, self.z_log_var, self.z], name='encoder')


  def build_decoder(self):
    latent_inputs = keras.layers.Input(shape=(self.latent_dim,), name='z_sampling')
    x = keras.layers.Dense(self.intermediate_dim, activation='relu')(latent_inputs)
    outputs = keras.layers.Dense(self.original_dim, activation='sigmoid')(x)

    # instantiate decoder model
    self.decoder = keras.models.Model(latent_inputs, outputs, name='decoder')

  def add_loss(self):
    # VAE loss = mse_loss or xent_loss + kl_loss
    reconstruction_loss = keras.losses.mse(self.inputs, self.outputs)
    reconstruction_loss *= self.original_dim

    kl_loss = 1 + self.z_log_var - K.square(self.z_mean) - K.exp(self.z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5

    vae_loss = K.mean(reconstruction_loss + kl_loss)

    self.vae.add_loss(vae_loss)

# WandB and Callbacks

In [0]:
class ReconstructionLogger(tf.keras.callbacks.Callback):
  def __init__(self, batch_size):
    self.batch_size = batch_size

    super(ReconstructionLogger, self).__init__()

  def on_epoch_end(self, logs, epoch):
    sample_images = test_img[:self.batch_size]
    
    images = []
    reconstructions = []
    
    for i in range(32):
        reconstruction = self.model.predict(sample_images[i].reshape((1,)+sample_images[i].shape))

        images.append(sample_images[i].reshape(28,28))
        reconstructions.append(reconstruction.reshape(28,28))

    wandb.log({"images": [wandb.Image(image)
                          for image in images]})
    wandb.log({"reconstructions": [wandb.Image(reconstruction)
                          for reconstruction in reconstructions]})

# Sweep Train 

In [0]:
def train():
    # Initialize wandb with a sample project name
    wandb.init(entity='ayush-thakur', project="keras-gan")
    
    (X_train, y_train) = train_img, train_label
    (X_test, y_test) = test_img, test_label

    # Specify the hyperparameter to be tuned along with
    # an initial value
    configs = {
        'latent_space': 2
    }
    
    # Specify the other hyperparameters to the configuration
    config = wandb.config
    config.batch_size = 32
    config.epochs = 10
    
    # Add the config item (layers) to wandb
    if wandb.run:
        wandb.config.update({k: v for k, v in configs.items() if k not in dict(wandb.config.user_items())})
        configs = dict(wandb.config.user_items())
    
    # Define the model
    vae = VariationalAutoencoder(input_shape=(784), 
                               original_dim=(784), 
                               intermediate_dim=512,
                               latent_dim=wandb.config.latent_space)
    model = vae.build_model()
    
    # Compile the model
    model.compile(optimizer='adam')
    
    # Train the model
    _ = model.fit(X_train,
              X_train, 
              epochs=config.epochs,
              batch_size=config.batch_size,
              callbacks=[WandbCallback(),
                         ReconstructionLogger(config.batch_size)])

In [0]:
sweep_config = {
  'method': 'grid',
  'parameters': {
      'latent_space': {
          'values': [2, 10, 100]
      }
  }
}

In [0]:
sweep_id = wandb.sweep(sweep_config, entity='ayush-thakur', project="keras-gan")

Create sweep with ID: jpa4wcus
Sweep URL: https://app.wandb.ai/ayush-thakur/keras-gan/sweeps/jpa4wcus


In [0]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: dpbmghtj with config:
	latent_space: 2
wandb: Agent Started Run: dpbmghtj


Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 45.8759
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 41.9013
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 40.9854
Epoch 4/10
1875/1875 [==============================] - 5s 2ms/step - loss: 40.3775
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 39.9124
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 39.5278
Epoch 7/10
1875/1875 [==============================] - 5s 2ms/step - loss: 39.1987
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 38.9309
Epoch 9/10
1875/1875 [==============================] - 5s 2ms/step - loss: 38.6799
Epoch 10/10
1875/1875 [==============================] - 5s 2ms/step - loss: 38.5134
wandb: Agent Finished Run: dpbmghtj 

wandb: Agent Starting Run: qs0ixe8m with config:
	latent_space: 10
wandb: Agent Started Run: qs0ixe8m


Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 38.3108
Epoch 2/10
1875/1875 [==============================] - 5s 2ms/step - loss: 32.6011
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 31.7226
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 31.2776
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 30.9187
Epoch 6/10
1875/1875 [==============================] - 5s 2ms/step - loss: 30.6243
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 30.3862
Epoch 8/10
1875/1875 [==============================] - 5s 2ms/step - loss: 30.1878
Epoch 9/10
1875/1875 [==============================] - 5s 2ms/step - loss: 30.0254
Epoch 10/10
1875/1875 [==============================] - 5s 3ms/step - loss: 29.8815
wandb: Agent Finished Run: qs0ixe8m 

wandb: Agent Starting Run: 44vn6ipu with config:
	latent_space: 100
wandb: Agent Started Run: 44vn6ipu


Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 43.0656
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 35.7577
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 33.3165
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 32.4275
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 31.9325
Epoch 6/10
1875/1875 [==============================] - 5s 2ms/step - loss: 31.5640
Epoch 7/10
1875/1875 [==============================] - 5s 2ms/step - loss: 31.2544
Epoch 8/10
1875/1875 [==============================] - 5s 2ms/step - loss: 31.0762
Epoch 9/10
1875/1875 [==============================] - 5s 2ms/step - loss: 30.8869
Epoch 10/10
1875/1875 [==============================] - 5s 3ms/step - loss: 30.7214
wandb: Agent Finished Run: 44vn6ipu 

